### Comments

* On a rajouté des features --> e.g. summary, production company, languages
* Batch??
* fine tuning des params --> pas optimisé pour le moment
* comprendre si les labels imbalance est problematique ????
* we use only cast crew for graph construction 


Question : 
* loss jump ?!
* stability of scores ?

skip connection
batch norm
self loop

# Machine Learning

In [13]:
import numpy as np
import pandas as pd
import pickle

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import f1_score, recall_score
from sklearn.utils import resample
from sklearn.decomposition import PCA

import random

import dgl.function as fn
from dgl import DGLGraph
import dgl.nn.pytorch as dgl_nn
import dgl.transform as dgl_transform

import time

import torch
import torch.nn as nn
import torch.nn.functional as F

Data_path = 'Data/'

### Loading the similarity matrix and generate graph

In [2]:
# Load similarity matrix
#file = open(Data_path + 'Adjacency_matrix.pickle','rb')
file = open(Data_path + 'Adjacency_matrix_Genre099_CC005.pickle','rb')
adj_mat =  pickle.load(file)

# Generate graph
G = DGLGraph(graph_data=adj_mat)
G = dgl_transform.add_self_loop(G)

### Loading features and labels

In [3]:
features_df = pd.read_csv(Data_path + 'features.csv',index_col=0).drop(columns = ['title'])

labels_df = pd.read_csv(Data_path + 'labels.csv',index_col=0).drop(columns = ['title'])

In [4]:
# features
tensor_feat = torch.FloatTensor(features_df.values)

# labels
IMDB_nom = labels_df['Nominations_Oscars'].copy()
IMDB_nom.loc[IMDB_nom > 0] = 1
tensor_labels = torch.LongTensor(IMDB_nom.values)

In [5]:
# Just checking class imbalance
IMDB_nom.value_counts() # 18.263 % of CLASS 1

0.0    3925
1.0     877
Name: Nominations_Oscars, dtype: int64

In [6]:
'''# Separate majority and minority classes
df_majority = IMDB_nom[IMDB_nom==0]
df_minority = IMDB_nom[IMDB_nom==1]
 
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=877,     # to match minority class
                                 random_state=1) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
 
# Display new class counts
df_downsampled.value_counts()
# 1    49
# 0    49
# Name: balance, dtype: int64

adj_mat_bal = adj_mat[df_downsampled.index,:]
adj_mat_bal = adj_mat_bal[:,df_downsampled.index]
G = DGLGraph(graph_data=adj_mat_bal)

G = dgl_transform.add_self_loop(G)

tensor_feat = torch.FloatTensor(features_df.iloc[df_downsampled.index,:].values)
tensor_labels = torch.LongTensor(df_downsampled.values)'''

'# Separate majority and minority classes\ndf_majority = IMDB_nom[IMDB_nom==0]\ndf_minority = IMDB_nom[IMDB_nom==1]\n \n# Downsample majority class\ndf_majority_downsampled = resample(df_majority, \n                                 replace=False,    # sample without replacement\n                                 n_samples=877,     # to match minority class\n                                 random_state=1) # reproducible results\n \n# Combine minority class with downsampled majority class\ndf_downsampled = pd.concat([df_majority_downsampled, df_minority])\n \n# Display new class counts\ndf_downsampled.value_counts()\n# 1    49\n# 0    49\n# Name: balance, dtype: int64\n\nadj_mat_bal = adj_mat[df_downsampled.index,:]\nadj_mat_bal = adj_mat_bal[:,df_downsampled.index]\nG = DGLGraph(graph_data=adj_mat_bal)\n\nG = dgl_transform.add_self_loop(G)\n\ntensor_feat = torch.FloatTensor(features_df.iloc[df_downsampled.index,:].values)\ntensor_labels = torch.LongTensor(df_downsampled.values)'

### Generate masks and split train, val, test

In [7]:
sss = StratifiedShuffleSplit(n_splits=1, train_size=0.8, random_state=0)

for prov_index, test_index in sss.split(tensor_feat, tensor_labels):
    train_mask = prov_index
    val_mask = test_index

#for train_index, val_index in sss1.split(tensor_feat[prov_mask], tensor_labels[prov_mask]):
#    train_mask = train_index
#    val_mask = val_index

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1788: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


### Building Graph Neural Network

In [29]:
class Graph_NN(nn.Module):
    
    def __init__(self,
                 in_feats: int,
                 out_feats: int,
                 k: int,
                 dropout_prob: float,
                 hidden_size: int):
        
        super().__init__()
        
        self._in_feats = in_feats
        self._out_feats = out_feats
        self._k = k
        self.hidden_size = hidden_size
        self.dropout = nn.Dropout(p=dropout_prob)
        
        # Layers
        #self.linear1 = nn.Linear(self._in_feats,self.hidden_size)
        self.appnpconv = dgl_nn.conv.APPNPConv(self._k,0.1,0) #alpha teleport proba = TA a dit 0.1
        self.linear2 = nn.Linear(self._in_feats,self._out_feats)
        
        
    def forward(self, graph, feat):
        
        #h = self.linear1(feat)
        h = self.appnpconv(graph,feat)
        h = F.relu(h)
        h = self.linear2(h)
        h = F.softmax(h)
        
        return h 


In [30]:
def train(model, g, features, labels, loss_fcn, train_mask, optimizer):
    
    model.train()  
    
    pred = model(g, features) # prediction
    loss = loss_fcn(pred[train_mask],labels[train_mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss

def evaluate(model, g, features, mask, labels):
    model.eval()  
    
    with torch.no_grad():
        pred = model(g, features)[mask]  # only compute the evaluation set
        labels = labels[mask]
        _, indices = torch.max(pred, dim=1)
        correct = torch.sum(indices == labels)
        acc = correct.item() * 1.0 / len(labels) #not the best metric
        f1 = f1_score(labels, indices)
        
        return f1,acc

In [31]:
# PCA
reduced_feats = PCA(n_components = 500).fit_transform(tensor_feat)
tensor_feat = torch.FloatTensor(reduced_feats)

In [34]:
# Fixed values
in_feats = tensor_feat.shape[1]
output = 2

# Not relevant parameters
n_epochs = 100

# To tune
k = 5 # what does this mean? Number of iterations??
lr = 0.01  # SMALL?
weight_decay = 5e-6
p_dropout = 0 # for now not doing it

hidden_size = 250
#weights_loss = torch.FloatTensor([0.15,0.85]) # to rebalance

In [35]:
model = Graph_NN(in_feats, output, k, p_dropout, hidden_size)

#loss_fcn = torch.nn.CrossEntropyLoss(weights_loss)
loss_fcn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=lr,
                             weight_decay=weight_decay)
dur = []
for epoch in range(n_epochs):
    if epoch >= 3:
        t0 = time.time()
    loss = train(model, G, tensor_feat, tensor_labels, loss_fcn, train_mask, optimizer)
    if epoch >= 3:
        dur.append(time.time() - t0)
        
    f1, acc = evaluate(model, G, tensor_feat, val_mask, tensor_labels)
    
    if epoch%1 == 0:
        print("Epoch {:05d} | Time(s) {:.4f} | Train Loss {:.4f} | Val f1 {:.4%} | Val Accuracy {:.4%}". format(
                epoch+1, np.mean(dur), loss.item(), f1, acc))

print()
f1, acc = evaluate(model, G, tensor_feat, test_mask, tensor_labels)
print("Test Accuracy {:.4%}".format(acc))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 00001 | Time(s) nan | Train Loss 0.6859 | Val f1 24.0664% | Val Accuracy 61.9543%
Epoch 00002 | Time(s) nan | Train Loss 0.6789 | Val f1 21.2766% | Val Accuracy 61.5385%
Epoch 00003 | Time(s) nan | Train Loss 0.6605 | Val f1 20.0957% | Val Accuracy 65.2807%
Epoch 00004 | Time(s) 18.9383 | Train Loss 0.6312 | Val f1 20.0000% | Val Accuracy 65.0728%
Epoch 00005 | Time(s) 19.1473 | Train Loss 0.6265 | Val f1 19.9052% | Val Accuracy 64.8649%
Epoch 00006 | Time(s) 18.9600 | Train Loss 0.6255 | Val f1 19.7183% | Val Accuracy 64.4491%
Epoch 00007 | Time(s) 18.8525 | Train Loss 0.6245 | Val f1 19.5349% | Val Accuracy 64.0333%
Epoch 00008 | Time(s) 18.7769 | Train Loss 0.6257 | Val f1 19.4444% | Val Accuracy 63.8254%
Epoch 00009 | Time(s) 18.7280 | Train Loss 0.6294 | Val f1 29.7787% | Val Accuracy 27.4428%
Epoch 00010 | Time(s) 18.6905 | Train Loss 1.0240 | Val f1 29.7189% | Val Accuracy 27.2349%
Epoch 00011 | Time(s) 18.6720 | Train Loss 1.0235 | Val f1 31.0000% | Val Accuracy 42.6195%


KeyboardInterrupt: 

In [ ]:
torch.sum(tensor_labels[val_mask]).item()/tensor_labels[val_mask].shape[0]